In the name of Allah

# tf-idf

In [16]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=9120b8e0ceec378419b6dc8d2c8c2930b8102bb8eb9c659096336ae3bffb331e
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180367 sha256=1e459a4f03b4ff70a1f0294e82df05752fb3d7f3f0d803993fafdbdb166ece54
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

In [17]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from __future__ import unicode_literals
from hazm import *

In [18]:
!cp /content/drive/MyDrive/MLSD/hw1/data/upsample_cat1_raw_data.csv /content/

In [19]:
df = pd.read_csv("/content/upsample_cat1_raw_data.csv")
df

,id,cat1,cat2,cat3,title,description,url,sub_title,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,10000000,-1,-1,-1,سایر,-1,غیره,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,864,vehicles,auto,car,پژو 206 SD V8 مدل 2006 بدون رنگ,سلام بفروشم SDV 8 تیوفایو مدل 86 آینه ها و شیش...,https://divar.ir/v/پژو-206-sd-v8-مدل-۱۳۸۶-بدون...,نیم ساعت پیش در تبریز,7200000000,-1,1381,-1,موکا,38000,Unknown,-1.0
2296,1428,vehicles,auto,car,پژو پارس LX TU5 مدل رندوم 1400,موتور سالم است جزئیات آگهی را با دقت بخوانید و...,https://divar.ir/v/پژو-پارس-lx-tu5-مدل-۱۴۰۰_سو...,نیم ساعت پیش در اصفهان شهرضا,660000000,-1,1387,-1,بژ,497000,Unknown,-1.0
2297,1321,vehicles,motorcycles,Unknown,موتور کلاسیک وسپا 170 سی سی دلتا,قیمت نقدی فروشگاه 83 میلیون، موتور 900 کیلومتر...,https://divar.ir/v/موتور-طرح-وسپا-کلاسیک-۱۷۰-س...,۱ ساعت پیش در مشهد فدک,550000000,-1,1375,-1,نقرآبی,276000,Unknown,-1.0
2298,91,vehicles,auto,car,Elantra 2018 کامل,فروش الانترا مدل 2018 فول وارداتی رادار جانبی ...,https://divar.ir/v/النترا-۲۰۱۸-فول_سواری-و-وان...,۱ ساعت پیش در شیراز، فرهنگ شهر,49500000,-1,1389,-1,Unknown,350000,Unknown,-1.0


In [20]:
text = list(df["title"])
text.extend(list(df["description"]))
text.extend(list(df["sub_title"]))

normalizer = Normalizer()
text_normalized = [[normalizer.normalize(y) for y in x.split()] for x in tqdm(text)]
text_sentences = [sent_tokenize(' '.join(x)) for x in tqdm(text_normalized)]
text_tokens = [[word_tokenize(sent) for sent in sents][0] for sents in tqdm(text_sentences)]

vectorizer = TfidfVectorizer(use_idf = True, norm ='l2', analyzer='word')
text_vector = vectorizer.fit_transform([' '.join(doc) for doc in text_tokens])

100%|██████████| 6900/6900 [00:00<00:00, 12666.96it/s]


In [23]:
title_encoding = [doc for doc in text_vector.toarray()[:2300]]
description_encoding = [doc for doc in text_vector.toarray()[2300:2300*2]]
sub_title_encoding = [doc for doc in text_vector.toarray()[2300*2:]]

In [24]:
df.insert(loc=6, column='description_encoding', value=description_encoding)
df.insert(loc=5, column='title_encoding', value=title_encoding)
df.insert(loc=10, column='sub_title_encoding', value=sub_title_encoding)

In [25]:
df

,id,cat1,cat2,cat3,title,title_encoding,description,description_encoding,url,sub_title,sub_title_encoding,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",10000000,-1,-1,-1,سایر,-1,غیره,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,864,vehicles,auto,car,پژو 206 SD V8 مدل 2006 بدون رنگ,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",سلام بفروشم SDV 8 تیوفایو مدل 86 آینه ها و شیش...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/پژو-206-sd-v8-مدل-۱۳۸۶-بدون...,نیم ساعت پیش در تبریز,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",7200000000,-1,1381,-1,موکا,38000,Unknown,-1.0
2296,1428,vehicles,auto,car,پژو پارس LX TU5 مدل رندوم 1400,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",موتور سالم است جزئیات آگهی را با دقت بخوانید و...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/پژو-پارس-lx-tu5-مدل-۱۴۰۰_سو...,نیم ساعت پیش در اصفهان شهرضا,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",660000000,-1,1387,-1,بژ,497000,Unknown,-1.0
2297,1321,vehicles,motorcycles,Unknown,موتور کلاسیک وسپا 170 سی سی دلتا,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",قیمت نقدی فروشگاه 83 میلیون، موتور 900 کیلومتر...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/موتور-طرح-وسپا-کلاسیک-۱۷۰-س...,۱ ساعت پیش در مشهد فدک,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",550000000,-1,1375,-1,نقرآبی,276000,Unknown,-1.0
2298,91,vehicles,auto,car,Elantra 2018 کامل,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",فروش الانترا مدل 2018 فول وارداتی رادار جانبی ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",https://divar.ir/v/النترا-۲۰۱۸-فول_سواری-و-وان...,۱ ساعت پیش در شیراز، فرهنگ شهر,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",49500000,-1,1389,-1,Unknown,350000,Unknown,-1.0


In [26]:
df.to_json("tfidf_em_data.json", orient="records")

In [27]:
!cp /content/tfidf_em_data.json /content/drive/MyDrive/MLSD/hw1/data/

# BERT embedding

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.4 MB/s eta 0:00:00


In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from transformers import AutoConfig, AutoTokenizer, AutoModel

In [2]:
!cp /content/drive/MyDrive/MLSD/hw1/data/normalized_raw_data.csv /content/

In [3]:
df = pd.read_csv("/content/normalized_raw_data.csv")
df

,id,cat1,cat2,cat3,title,description,url,sub_title,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,10000000,-1,-1,-1,سایر,-1,غیره,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1424,vehicles,auto,car,پژو پارس پرشیا اتومات فابریک موتور TU۵نقد اقساط,سلام و درود خدمت همه عزیزان نقد و اقساط حداقل ...,https://divar.ir/v/پژو-پارس-پرشیا-اتومات-فابری...,نیم ساعت پیش در تهران، نازی‌آباد,360000000,-1,1389,-1,خاکستری,143000,Unknown,-1.0
1424,1425,vehicles,auto,car,جیلی GC۶ اکسلنت، مدل ۱۳۹۸,خودرو در حد صفر میباشد اتوماتیک دارای گیربکس ژ...,https://divar.ir/v/جیلی-gc6-اکسلنت-مدل-۱۳۹۸_سو...,نیم ساعت پیش در تهران، پونک,740000000,-1,1398,-1,سفید,60000,Unknown,-1.0
1425,1426,vehicles,auto,car,ام وی ام ایکس۳۳ATنیوفیس جدید,ام وی ام ایکس۳۳ATنیوفیس بسیار تمیز و بدون خط و...,https://divar.ir/v/ام-وی-ام-ایکس۳۳-at-نیوفیس-ج...,نیم ساعت پیش در تهران، ولنجک,1190000000,-1,1400,-1,خاکستری,56000,Unknown,-1.0
1426,1427,vehicles,auto,car,پژو ۲۰۶ تیپ ۲، مدل ۱۳۹۸,فوق العاده سالم و بی رنگ یک جفت باند پایونیر چ...,https://divar.ir/v/پژو-206-تیپ-۲-مدل-۱۳۹۸_سوار...,نیم ساعت پیش در تهران، دریاچه شهدای خلیج فارس,468000000,-1,1398,-1,سفید,72000,Unknown,-1.0


In [ ]:
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
bert_model = AutoModel.from_pretrained(model_name_or_path)

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

In [ ]:
print('Encoding title')
title_encoding = []
for text in tqdm(df["title"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = bert_model(**encoded_input)
    title_encoding.append(output.pooler_output[0].detach().numpy())

title_encoding = [t.astype('float64') for t in title_encoding]
df.insert(loc=5, column='title_encoding', value=title_encoding)


print('Encoding description')
description_encoding = []
for text in tqdm(df["description"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = bert_model(**encoded_input)
    description_encoding.append(output.pooler_output[0].detach().numpy())

description_encoding = [t.astype('float64') for t in description_encoding]
df.insert(loc=7, column='description_encoding', value=description_encoding)


print('Encoding sub_title')
sub_title_encoding = []
for text in tqdm(df["sub_title"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = bert_model(**encoded_input)
    sub_title_encoding.append(output.pooler_output[0].detach().numpy())

sub_title_encoding = [t.astype('float64') for t in sub_title_encoding]
df.insert(loc=10, column='sub_title_encoding', value=sub_title_encoding)

Encoding title


100%|██████████| 1428/1428 [03:29<00:00,  6.80it/s]


Encoding description


100%|██████████| 1428/1428 [08:50<00:00,  2.69it/s]


Encoding sub_title


100%|██████████| 1428/1428 [02:38<00:00,  9.00it/s]


In [ ]:
df

,id,cat1,cat2,cat3,title,title_encoding,description,description_encoding,url,sub_title,sub_title_encoding,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,"[-0.18732066452503204, -0.16019654273986816, -...",با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,"[0.6496793031692505, 0.3636835813522339, 0.165...",https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,"[-0.6320223212242126, 0.602863609790802, 0.486...",14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,"[-0.26654475927352905, 0.10652824491262436, -0...",لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,"[0.5611435174942017, -0.12848199903964996, -0....",https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,"[-0.56597900390625, 0.6006038784980774, 0.6099...",18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,"[-0.4316917657852173, 0.11164090037345886, -0....",سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,"[0.3416299819946289, -0.49759653210639954, 0.0...",https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,"[-0.5275211930274963, 0.4701727330684662, 0.18...",3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,"[0.39403724670410156, 0.036260999739170074, -0...",بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,"[0.16852132976055145, -0.32285115122795105, 0....",https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,"[-0.5898498892784119, 0.5805869102478027, 0.45...",17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,"[-0.5452685356140137, 0.10621759295463562, -0....",با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,"[0.6299875378608704, 0.49222251772880554, 0.14...",https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,"[-0.5387448668479919, 0.36239051818847656, 0.3...",10000000,-1,-1,-1,سایر,-1,غیره,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1424,vehicles,auto,car,پژو پارس پرشیا اتومات فابریک موتور TU۵نقد اقساط,"[0.1496419608592987, 0.07308761030435562, -0.1...",سلام و درود خدمت همه عزیزان نقد و اقساط حداقل ...,"[0.48633256554603577, -0.34288686513900757, -0...",https://divar.ir/v/پژو-پارس-پرشیا-اتومات-فابری...,نیم ساعت پیش در تهران، نازی‌آباد,"[-0.3271942138671875, 0.47210150957107544, 0.2...",360000000,-1,1389,-1,خاکستری,143000,Unknown,-1.0
1424,1425,vehicles,auto,car,جیلی GC۶ اکسلنت، مدل ۱۳۹۸,"[-0.2679332494735718, 0.23268145322799683, 0.0...",خودرو در حد صفر میباشد اتوماتیک دارای گیربکس ژ...,"[-0.4382700026035309, 0.30906233191490173, 0.0...",https://divar.ir/v/جیلی-gc6-اکسلنت-مدل-۱۳۹۸_سو...,نیم ساعت پیش در تهران، پونک,"[-0.6441807746887207, 0.268715500831604, 0.185...",740000000,-1,1398,-1,سفید,60000,Unknown,-1.0
1425,1426,vehicles,auto,car,ام وی ام ایکس۳۳ATنیوفیس جدید,"[-0.14763480424880981, 0.2872667610645294, 0.0...",ام وی ام ایکس۳۳ATنیوفیس بسیار تمیز و بدون خط و...,"[0.2541711926460266, -0.19120347499847412, -0....",https://divar.ir/v/ام-وی-ام-ایکس۳۳-at-نیوفیس-ج...,نیم ساعت پیش در تهران، ولنجک,"[-0.6512414813041687, 0.3127753436565399, -0.0...",1190000000,-1,1400,-1,خاکستری,56000,Unknown,-1.0
1426,1427,vehicles,auto,car,پژو ۲۰۶ تیپ ۲، مدل ۱۳۹۸,"[-0.5176522731781006, 0.08604994416236877, 0.2...",فوق العاده سالم و بی رنگ یک جفت باند پایونیر چ...,"[0.37596243619918823, 0.5743122696876526, 0.36...",https://divar.ir/v/پژو-206-تیپ-۲-مدل-۱۳۹۸_سوار...,نیم ساعت پیش در تهران، دریاچه شهدای خلیج فارس,"[-0.5883391499519348, 0.38904958963394165, 0.1...",468000000,-1,1398,-1,سفید,72000,Unknown,-1.0


In [ ]:
df.to_json("bert_em_data.json", orient="records")

In [ ]:
!cp /content/bert_em_data.json /content/drive/MyDrive/MLSD/hw1/data/

# Fasttext

In [ ]:
!pip install fasttext

In [5]:
import fasttext.util
import pandas as pd
from tqdm import tqdm
import numpy as np

In [6]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz

--2023-03-30 07:05:15--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fa.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.165.83.79, 18.165.83.91, 18.165.83.44, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.165.83.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4502524724 (4.2G) [application/octet-stream]
Saving to: ‘cc.fa.300.bin.gz’

cc.fa.300.bin.gz    100%[===================>]   4.19G  29.3MB/s    in 2m 15s  

2023-03-30 07:07:30 (31.8 MB/s) - ‘cc.fa.300.bin.gz’ saved [4502524724/4502524724]



In [7]:
!gunzip /content/cc.fa.300.bin.gz

In [8]:
ft = fasttext.load_model('cc.fa.300.bin')

In [9]:
!cp /content/drive/MyDrive/MLSD/hw1/data/downsample_cat1_raw_data.csv /content/

In [10]:
df = pd.read_csv("/content/downsample_cat1_raw_data.csv")
df

,id,cat1,cat2,cat3,title,description,url,sub_title,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,10000000,-1,-1,-1,سایر,-1,غیره,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,723,real-estate,buy-commercial-property,buy-office,آپارتمان ۱۰۰ متر با موقعیت اداری,با سلام خدمت بازدید کنندگان محترم یکی از شیک‌ت...,https://divar.ir/v/آپارتمان-۱۰۰-متر-با-موقعیت-...,۱ هفته پیش در شیراز، فرهنگ شهر,8000000000,100,1401,2,Unknown,-1,Unknown,-1.0
1133,632,real-estate,buy-commercial-property,buy-office,واحد ۱۱۰ متری در خیابان حسنی,واحد کلید نخورده و دارای دارای ۲ عدد سرویس بهد...,https://divar.ir/v/واحد-۱۱۰-متری-در-خیابان-حسن...,۱ هفته پیش در ارومیه,3800000000,110,1401,0,Unknown,-1,Unknown,-1.0
1134,135,real-estate,buy-commercial-property,buy-office,آپارتمان با موقعیت اداری ۵۰متر,سلام. فروش فوری دفتر کار با موقعیت و قیمت عالی.,https://divar.ir/v/آپارتمان-با-موقعیت-اداری-۵۰...,۶ روز پیش در اصفهان، آذر، خیابان شمس آبادی (کو...,-1,50,1390,1,Unknown,-1,Unknown,-1.0
1135,1378,real-estate,buy-commercial-property,buy-office,۷۶متر یک خواب دفترکار، مطب، فرامرز,دفتر فوق دارای امکانات اداری است [[کابل سیستم ...,https://divar.ir/v/۷۶متر-یک-خواب-دفترکار-مطب-ف...,۲ روز پیش در مشهد، فرامرز عباسی,2850000000,76,1391,1,Unknown,-1,Unknown,-1.0


In [11]:
title_encoding = [ft.get_word_vector(t) for t in df['title']]
description_encoding = [ft.get_word_vector(t) for t in df['description']]
sub_title_encoding = [ft.get_word_vector(t) for t in df['sub_title']]

In [12]:
df.insert(loc=6, column='description_encoding', value=description_encoding)
df.insert(loc=5, column='title_encoding', value=title_encoding)
df.insert(loc=10, column='sub_title_encoding', value=sub_title_encoding)

In [13]:
df

,id,cat1,cat2,cat3,title,title_encoding,description,description_encoding,url,sub_title,sub_title_encoding,price,size,year,room_number,color,mileage,CPU,RAM
0,1,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک hp,"[0.0044191913, 0.0014105368, -0.003266572, 0.0...",با درود لپ تاپ‌های استوک مارک hp hp probook ۶۵...,"[0.0055834227, 0.0084018065, 0.0012574522, 0.0...",https://divar.ir/v/لپ-تاپ-استوک-hp_رایانه-همرا...,۲ ساعت پیش در شیراز، ملاصدرا,"[-0.0017112222, 0.02765921, 0.010748463, 0.012...",14900000,-1,-1,-1,خاکستری,-1,Core i7,8.0
1,2,electronic-devices,computers,laptop-notebook-macbook,لپتاپ HP PRO نسل ۹ مهندسی,"[0.032628242, 0.034881044, 0.011885938, 0.0091...",لپ تاپ، مهندسی، گرافیکی، پر سرعت شیک و ظریف بر...,"[0.008914363, 0.010127283, 0.0022957255, 0.002...",https://divar.ir/v/لپتاپ-hp-pro-نسل-۹-مهندسی_ر...,۲ ساعت پیش در شیراز، وصال,"[0.00692525, 0.026805883, 0.008052798, 0.01132...",18800000,-1,-1,-1,نقره‌ای,-1,Ryzen 5,8.0
2,3,electronic-devices,computers,laptop-notebook-macbook,فروش لپتاپ استوک اروپایی,"[0.0380223, 0.037487496, -0.0030205941, 0.0176...",سلام تکنو رایان واردکننده لپتاپ استوک اروپایی ...,"[0.0066714115, 0.015034727, 0.0036566602, 0.00...",https://divar.ir/v/فروش-لپتاپ-استوک-اروپایی_را...,۳ ساعت پیش در شیراز، زند,"[-0.010540829, 0.020279286, 0.007543174, 0.012...",3900000,-1,-1,-1,Unknown,-1,Unknown,4.0
3,4,electronic-devices,computers,laptop-notebook-macbook,HP ZBOOK۱۵ g۳ با گرافیک مجزا مناسب طراحی,"[0.024284165, 0.036271878, 0.0059037153, 0.007...",بدون واسطه به قیمت عمده خرید کنید فروشگاه لپت...,"[0.0067295157, 0.008296178, 0.0028628968, 0.00...",https://divar.ir/v/hp-zbook15-g3-با-گرافیک-مجز...,۳ ساعت پیش در شیراز، ملاصدرا,"[-0.00706585, 0.030991036, 0.012722894, 0.0145...",17900000,-1,-1,-1,مشکی,-1,Core i7,8.0
4,5,electronic-devices,computers,laptop-notebook-macbook,لپ تاپ استوک واردات مستقیم از دبی,"[0.020904575, -0.0019874193, -0.00658881, 0.01...",با سلام و احترام خدمت شما کاربر گرامی. کمال ت...,"[0.007092952, 0.01360633, 0.001662748, 0.00313...",https://divar.ir/v/لپ-تاپ-استوک-واردات-مستقیم-...,۳ ساعت پیش در شیراز، پودنک,"[-0.006828717, 0.026144609, -1.6363028e-05, 0....",10000000,-1,-1,-1,سایر,-1,غیره,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,723,real-estate,buy-commercial-property,buy-office,آپارتمان ۱۰۰ متر با موقعیت اداری,"[0.028177477, 0.01715723, 0.0032611329, -0.000...",با سلام خدمت بازدید کنندگان محترم یکی از شیک‌ت...,"[0.004517796, 0.0012675273, 0.003930048, 0.005...",https://divar.ir/v/آپارتمان-۱۰۰-متر-با-موقعیت-...,۱ هفته پیش در شیراز، فرهنگ شهر,"[0.00063945923, 0.010722921, 0.009166466, 0.00...",8000000000,100,1401,2,Unknown,-1,Unknown,-1.0
1133,632,real-estate,buy-commercial-property,buy-office,واحد ۱۱۰ متری در خیابان حسنی,"[0.030709827, 0.03167705, -0.006713265, -0.001...",واحد کلید نخورده و دارای دارای ۲ عدد سرویس بهد...,"[0.0074826013, 0.014009332, 0.00031760626, 0.0...",https://divar.ir/v/واحد-۱۱۰-متری-در-خیابان-حسن...,۱ هفته پیش در ارومیه,"[0.015245804, 0.00048380822, -0.01041408, -0.0...",3800000000,110,1401,0,Unknown,-1,Unknown,-1.0
1134,135,real-estate,buy-commercial-property,buy-office,آپارتمان با موقعیت اداری ۵۰متر,"[0.02618922, 0.011727185, 0.0006198676, 0.0087...",سلام. فروش فوری دفتر کار با موقعیت و قیمت عالی.,"[0.013945272, 0.0064696623, -0.0027345666, 0.0...",https://divar.ir/v/آپارتمان-با-موقعیت-اداری-۵۰...,۶ روز پیش در اصفهان، آذر، خیابان شمس آبادی (کو...,"[0.008927375, 0.0040322854, 0.007949539, 0.002...",-1,50,1390,1,Unknown,-1,Unknown,-1.0
1135,1378,real-estate,buy-commercial-property,buy-office,۷۶متر یک خواب دفترکار، مطب، فرامرز,"[7.7813165e-05, 0.0071415175, 0.0050706393, 0....",دفتر فوق دارای امکانات اداری است [[کابل سیستم ...,"[0.015840756, 0.016628738, -0.0018031221, 0.00...",https://divar.ir/v/۷۶متر-یک-خواب-دفترکار-مطب-ف...,۲ روز پیش در مشهد، فرامرز عباسی,"[0.0124428915, 0.007608268, 0.0006387339, 0.00...",2850000000,76,1391,1,Unknown,-1,Unknown,-1.0


In [14]:
df.to_json("fasttext_em_data.json", orient="records")

In [15]:
!cp /content/fasttext_em_data.json /content/drive/MyDrive/MLSD/hw1/data/